In [2]:
# import sqlite3
# conn = sqlite3.connect('../database.db')
# c = conn.cursor()
# c.execute('''DROP TABLE PlayerStats''')
# conn.commit()

In [1]:
# import sqlite3
# conn = sqlite3.connect('../database.db')
# c = conn.cursor()
# c.execute('''DROP TABLE TeamStats''')
# conn.commit()

In [6]:
import sqlite3
conn = sqlite3.connect('../database.db')
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('PlayerStats',), ('TeamStats',)]


# PlayerStats

In [4]:
import os
import sqlite3

conn = sqlite3.connect('../database.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS PlayerStats (
            season_id INTEGER NOT NULL,
            player_id INTEGER NOT NULL,
            game_id INTEGER NOT NULL,
            game_date TEXT NOT NULL,
            opponent_team_id INTEGER NOT NULL,
            minutes INTEGER,
            home_game BOOLEAN,
            points INTEGER,
            field_goals_made INTEGER,
            field_goals_attempted INTEGER,
            three_point_field_goals_made INTEGER,
            three_point_field_goals_attempted INTEGER,
            free_throws_made INTEGER,
            free_throws_attempted INTEGER,
            offensive_rebounds INTEGER,
            defensive_rebounds INTEGER,
            total_rebounds INTEGER,
            assists INTEGER,
            steals INTEGER,
            blocks INTEGER,
            turnovers INTEGER,
            PRIMARY KEY (season_id, player_id, game_id)
        );''')
conn.commit()

In [6]:
import os
import sqlite3

conn = sqlite3.connect('../database.db')
c = conn.cursor()
c.execute('''SELECT opponent_team_id from PlayerStats ORDER by LIMIT 10''')
print(c.fetchall())
conn.commit()

[(1610612764,), (1610612751,), (1610612754,), (1610612755,), (1610612752,), (1610612765,), (1610612748,), (1610612753,), (1610612760,), (1610612740,)]


In [19]:
import os
import sqlite3
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players

players_list = players.find_players_by_full_name('James Harden')
pid  = players_list[0]['id']
conn = sqlite3.connect('../database.db')
c = conn.cursor()
c.execute(f'''SELECT game_date, points from PlayerStats WHERE player_id={pid} ORDER BY game_date  DESC LIMIT 5''')
print(c.fetchall())
conn.commit()

[('2023-11-25', 8), ('2023-11-24', 8), ('2023-11-22', 16), ('2023-11-20', 13), ('2023-11-17', 24)]


# TeamStats

In [5]:
import os
import sqlite3

conn = sqlite3.connect('../database.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS TeamStats (
            game_id INTEGER NOT NULL,
            team_id INTEGER NOT NULL,
            opponent_team_id INTEGER NOT NULL,
            game_date TEXT NOT NULL,
            home_game BOOLEAN NOT NULL,
            defensive_rating REAL NOT NULL,
            effective_defensive_rating REAL NOT NULL,
            pace REAL NOT NULL,
            opponent_score INTEGER NOT NULL,
            opponent_field_goals_made INTEGER NOT NULL,
            opponent_field_goals_attempted INTEGER NOT NULL,
            opponent_three_point_field_goals_made INTEGER NOT NULL,
            opponent_three_point_field_goals_attempted INTEGER NOT NULL,
            opponent_free_throws_made INTEGER NOT NULL,
            opponent_free_throws_attempted INTEGER NOT NULL,
            opponent_defensive_rebounds INTEGER NOT NULL,
            opponent_offensive_rebounds INTEGER NOT NULL,
            opponent_total_rebounds INTEGER NOT NULL,
            opponent_assists INTEGER NOT NULL,
            opponent_steals INTEGER NOT NULL,
            opponent_blocks INTEGER NOT NULL,
            opponent_turnovers INTEGER NOT NULL,
            PRIMARY KEY (team_id, game_date)
        );''')
conn.commit()

In [3]:
import os
import sqlite3
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import teams

team_list = teams.find_teams_by_full_name('Houston Rockets')
tid = team_list[0]['id']
conn = sqlite3.connect('../database.db')
c = conn.cursor()
c.execute(f'''SELECT pace from TeamStats''')
data = c.fetchall()
conn.commit()

In [4]:
import numpy as np

mean = np.mean(data)
std = np.std(data)
normalize_data = (data - mean) / std

99.88934426229508

In [35]:
import os
import sqlite3
from nba_api.stats.static import players

players_list = players.find_players_by_full_name('James Harden')
pid  = players_list[0]['id']
conn = sqlite3.connect('../database.db')
c = conn.cursor()
# c.execute(f'''SELECT PlayerStats.game_date, PlayerStats.points, TeamStats.pace 
#           from TeamStats, PlayerStats 
#           WHERE PlayerStats.opponent_team_id = TeamStats.team_id 
#           AND PlayerStats.game_id = TeamStats.game_id
#           AND PlayerStats.player_id={pid}
#           ORDER BY PlayerStats.game_date 
#           DESC LIMIT 5''')

c.execute(f'''
    SELECT 
        PS.game_date, 
        PS.points, 
        AVG(TS.opponent_score) OVER (
            ORDER BY PS.game_date 
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) as avg_pace_up_to_game
    FROM 
        PlayerStats PS
    JOIN 
        TeamStats TS ON PS.opponent_team_id = TS.team_id AND PS.game_id = TS.game_id
    WHERE 
        PS.player_id = {pid}
    ORDER BY 
        PS.game_date DESC 
    LIMIT 5;
''')
data = c.fetchall()
conn.commit()

In [36]:
print(data)

[('2023-11-25', 8, 107.7), ('2023-11-24', 8, 107.77777777777777), ('2023-11-22', 16, 108.0), ('2023-11-20', 13, 107.85714285714286), ('2023-11-17', 24, 105.16666666666667)]
